# 4. Regex

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## Tartalom

* [1. Elmélet](#1.-Elmélet)
* [2. Megvalósítás](#2.-Megvalósítás)
    * [2.1. Szintaxis](#2.1.-Szintaxis)
    * [2.2. Gyakorlatok](#2.2.-Gyakorlatok)
* [3. Python](#3.-Python)
    * [3.1. Fontosabb függvények](#3.1.-Fontosabb-függvények)
    * [3.2. Mohóság](#3.2.-Mohóság)
    * [3.3. lookahead és lookbehind](#3.3.-lookahead-és-lookbehind)
    * [3.4. Gyakorlatok](#3.4.-Gyakorlatok)
* [4. Irodalom](#4.-Irodalom)

## 1. Elmélet

$\sum$ : az ábécé (karakterek nem üres, véges halmaza)

$string$: $\sum$ karaktereiből álló véges sorozat

$\varepsilon$: üres string

$\sum^\ast$ : $\sum$ feletti összes string halmaza ($\sum$ Kleene-lezárása)

$nyelv$: $\sum^\ast$ egy részhalmaza

$reguláris\ kifejezés$: Stringek egy halmazát határozza meg (egy *reguláris nyelvet*). Tartalmazhat literális karaktereket ill. az alábbi műveleti jeleket.

Műveletek:

* Konkatenáció: $R^1$ és $R^2$ reguláris kifejezés, ekkor $R^1R^2$ is regularis kifejezés és $R^1R^2 = \{\alpha\beta \ :\ \alpha \in R^1\ és\ \beta \in R^2\}$.
* Unió: $R^1$ és $R^2$ reguláris kifejezés, ekkor $R^1\mid R^2$ is regularis kifejezés és $R^1\mid R^2 = \{\alpha\ :\ \alpha \in R^1\ vagy\ \alpha \in R^2\}$.
* Kleene csillag: $R$ reguláris kifejezés, ekkor $R^\ast$ is regularis kifejezés és, mely tartalmazza az üres stringet ($\varepsilon$) és $R$ elemeinek tetszőleges konkatenációját.

## 2. Megvalósítás

Több szabványos "nyelvjárás" létezik (de ezeken túl az egyes programok is működhetnek különbözőképpen):
* *basic* (BRE): kerek és kapcsos zárójelek eszképelve, nincs `+`, `?` és `|`
* *extended* (ERE): ezt fogjuk tanulni
* *perl-kompatibilis* (PCRE): lustaság és sok minden más (Python-ban is ilyesmi van)

A *grep* és a *sed* alapból a BRE-t használják.
* *sed -r* vagy *sed -E*: kiterjesztett regularis kifejezések (ERE)  használata
* *grep -E* vagy *egrep*: kiterjesztett regularis kifejezések (ERE) használata
* *grep -P*: PCRE használata

### 2.1. Szintaxis

Pozícióra (nulla karakterre) illesztés:

* `^`: string / sor elejére illeszkedik
* `$`: string / sor végére illeszkedik

Egy karakterre illesztés:

* `.`: bármilyen karakterre illeszkedik
* különleges karakterre azt iszképelve lehet illeszteni, pl. `\.` illeszkedik a *.*-ra.
* `x`: literális karakter, saját magára illeszkedik
* `[ ]`: a zárójelen belül felsorolt karakterek valamelyikére illeszkedik, pl. `[ab]` illeszkedik az `a` vagy a `b` karakterre, másra nem.
    * Megadható tartomány is, pl `[a-z]` illeszkedik az ASCII kisbetűkre, `[0-9]` pedig a számjegyekre.
    * Ha a kötőjelet is be akarjuk venni a felsorolt karakterek közé, akkor a felsorolás elejére vagy végére kell írni.
    * Ha szögletes záró zárójelet akarunk a felsorolásba foglalni, akkor azt közvetlenül a nyitó után kell tenni.
    * A szögletes zárójelen belül más karakterek elveszítik speciális jelentésüket, pl. `[.]` egy literális pontra illeszkedik, nem pedig bármire.
* `[^ ]` illeszkedik a zárójelen belül fel nem sorolt karakterek valamelyikére. Megadható tartomány is, pl. [^A-Z0-9] illeszkedik minden karakterre, ami nem ASCII nagybetű és nem is számjegy.

Változó hosszúságú illesztések:

* `|`: Alternáció, az előtte vagy az utána következő regex valamelyikére illeszkedik, pl. `abcd|xyz` illeszkedik *abcd*-re és *xyz*-re is.
* `()`: A zárójelen belüli kifejezés megnevezett csoport lesz, amire később hivatkozni lehet. Általában egymásba ágyazhatók, de nem fedhetnek át. Pl `(a.(.a))` illeszkedik az *abba* stringre.
* `\n`: Hivatkozás egy csoportra. Pl. `(a.(.a)) \2 \1` illeszkedik az *abba ba abba* stringre.
* `?`: nulla vagy egy az előző karakterből / csoportból
* `*`: nulla vagy bármennyi az előző karakterből / csoportból
* `+`: legalább egy az előző karakterből / csoportból
* `{m,n}`: minimum *m*, maximum *n* darab az előző karakterből / csoportból.
    * `{m,}` alakban csak a minimumot is megadhatjuk (a maximum ekkor bármennyi lehet, hasonlóan a `*`-hoz).
    * `{,n}` alakban csak a maximumot is megadhatjuk (a minimum ekkor nulla, hasonlóan a `*`-hoz)

Műveletek precedenciája

  *csillag > konkatenáció > alternáció*

POSIX karakterosztályok

* `[:alpha:]`: betűk
* `[:lower:]`: kisbetűk
* `[:upper:]`: nagybetűk
* `[:alnum:]`: betűk és számjegyek
* `[:digit:]`: számjegyek
* `[:blank:]`: space és TAB
* `[:space:]`: whitespace-ek (`[ \t\r\n\v\f]`)
* `[:punct:]`: írásjelek (`[][!"#$%&'()*+,./:;<=>?@\^_`{|}~-]`)

Egyéb karakterosztályok

* `\s`: whitespace karakterek
* `\S`: nem whitespace karakterek
* `\w`: szóalkotó karakterek (számjegyek, betűk és alulvonás)
* `\W`: nem szóalkotó karakterek

Egy-egy reguláris nyelv általában sokféleképpen megadhatók regexekkel (pl. `a+` = `aa*`), nincs igazán jó egyszerűsítő / normálformára hozó algoritmus. $\to$ Érdemes jól megírni a regexeket!

### 2.2. Gyakorlatok

* Hány szó kezdődik *a*-val a `train/stopwords.txt` listában?
* Hány három betűs szó van a listában?
* Írjunk *grep* parancsot, ami alkalmas egy szövegben található szóismétlések szűrésére!
* Írjunk *sed* parancsot, ami a bemenetének minden szavát megismételteti! (pl. *Hát maga megbolondult?* $\to$ *Hát Hát maga maga megbolondult megbolondult?*)
* A nagy egymásra épülő gyakorlat sorozat:
    1. Alakítsuk át a `train/kant.txt` szöveget: egy sor = egy szó, punktuációkat dobjuk ki, üres sorokat töröljük.
    2. A kapott kimenetet alakítsuk tovább szógyakorisági listává: egy sor egy szót és az ő gyakoriságát tartalmazza. Egy szó (type) csak egyszer forduljon elő. A lista gyakoriság szerint fordítottan legyen rendezve.
    3. A szógyakorisági listát szűrjük a `stopwords-full.txt`-vel

In [ ]:
# gyakorisági adatok (előfordulások)
#!sed -r 's/\s+/\n/g' train/kant.txt | sed -r 's/\W+//g' | egrep -v '^$|[0-9]+' | egrep -x -v -i -f train/stopwords.txt | sort | uniq -c | sort -nr | sed -r "s/^ +//" | cut -d" " -f1 | uniq -c
#freq = !sed -r 's/\s+/\n/g' train/kant.txt | sed -r 's/\W+//g' | egrep -v '^$|[0-9]+' | egrep -x -v -i -f train/stopwords.txt | sort | uniq -c | sort -nr | sed -r "s/^ +//" | cut -d" " -f1 | uniq -c

# stringből számok kinyerése
#freq = [x.strip().split(' ') for x in freq]
#freq = [(int(y), int(x)) for x,y in freq]
#print(freq)
#[print(x, y, sep='\t') for x, y in freq]

# relatív gyakoriág
#wordcount = sum([x*y for x, y in freq])
#print('word count:', wc)
#freq = [(x/wc, y) for x,y in freq]
#print('freq', 'type', sep='\t')
#[print(round(x, 4), y, sep='\t') for x, y in freq]

# plottolás
#import matplotlib.pyplot as plt
#plt.plot([x[0] for x in zipf], [x[1] for x in zipf])
#plt.xlabel('frequency of types')
#plt.ylabel('number of types')
#plt.show()

## 3. Python

Python-ban a *re* modulban találjuk a regex-ekkel kapcsolatos dolgokat.

In [ ]:
import re

**Raw string-ek**

A Python hajlamos a stringekben lecserélni dolgokat (pl. a `\n`-t sortörésre, stb.) hogy ne kelljen mindent kiiszképelni (sőt, az iszképelő backslash-t is iszképelni), ezért regex-eknél érdemes *raw string*-eket használni, ezeket nem változtatja meg a Python:

```python
p = r'valami'
p = r"valami más"
```

### 3.1. Fontosabb függvények

#### 3.1.1. Keresés

`re.search(pattern, text)  # mit, miben`

A megadott szövegben keresi a mintára való első illeszkedést (a `re.match()` és a `re.fullmatch()` megszorítottabb: csak a szöveg eleji, ill. a teljes szövegre való illeszkedést fogadja el).
Egy ún. *match object*-tel tér vissza, amiben a találatok vannak.

In [ ]:
m = re.search(r'(\w+) (\w+) (\d+)?', 'ritkán rikkant a rigó')
if m:
    print(m.group(0)) # a teljes illeszkedő szövegrész (nem kell zárójelezve legyen)
    print(m.group(1)) # 1-as csoport
    print(m.group(2)) # 2-es csopot
    print(m.group(3)) # 3-es csopot
    print(m.group(1,2)) # tuple az 1-es és a 2-es csoportból

`re.findall(pattern, text)  # mit, miben`

Az összes mintára illeszkedő (és át nem fedő) szövegrész listáját adja vissza.

In [ ]:
# miért nincs 'eggel' vagy 'itkán'?

m = re.findall('\w{5}', 'korán reggel ritkán')
print(m)

#### 3.1.2. Csere és törlés

`re.sub(pattern, replace, text)  # mit, mire, miben`

Helyettesítés: a szövegben (*miben*) a mintára (*mit*) illeszkedő részeket lecseréli (*mire*)

In [ ]:
new = re.sub(r'(\d+)', r'\1 db', '10 rigó rikkant')
print(new)

#### 3.1.3. Flag-ek

A fenti függvényeknek opcionálisan megadhatók ún. flag-ek, amikkel módosítani lehet ezek működését (pl. `re.search(pattern, text, flag)`). Több flag is használható egyidejűleg, ekkor a `|` (bitenkénti vagy) operátorral kell ezeket összekötni.

`re.ASCII`

Python3-ban a minták unicode-minták, pl. a `\w` bármilyen unicode betűre illeszkedik, nem csak az ASCII betűkre (`[A-Za-z]`). Ha azt akarjuk, hogy a csak ASCII karakterkre illeszkedjenek a mintáink, akkor használjuk a `re.ASCII`-t.

Megjegyzés: a mintákat és a szövegeket is meg lehet adni *bytes*-ban, ekkor nincs szükség a `re.ASCII`-ra.

`re.IGNORECASE`

Hatására a kis- és nagybetű különbség figyelmen kívül hagyatik, pl. a `[a-z]` egyaránt illeszkedik a kis- és a nagybetűkre.

#### 3.1.4. Optimalizálás

`re.compile(pattern)`

A mintából mindig egy objektum generálodik. A *compile()*-lal elmenthetjük ezt az objektumot, ami tetszőleges függvénnyel használható a továbbiakban. Példa:

```python
pat = re.compile(pattern)
pat.search(text)
```

ugyanaz, mint a
```python
re.search(pattern, text)
```

Az objektum-generálás elég erőforrásigényes, ezért ha többször használunk egy regex-et (pl. egy for cikluson belül), akkor mindenképpen érdemes kompillálni, különben jóval lassabban fog futni.

In [ ]:
%%timeit

pat = re.compile(r'a(.)\1a')
for i in range(100):
    pat.search('az abbát kéne abbahagyni')

In [ ]:
%%timeit

for i in range(100):
    re.search(r'a(.)\1a', 'az abbát kéne abbahagyni')

### 3.2. Mohóság

Mohó operátorok:

* `?`
* `*`
* `+`

Lusta operátorok:

* `??`
* `*?`
* `+?`

In [ ]:
import re

string = 'xaaaxaaax'
greedy = re.compile(r'x.*x')
lazy = re.compile(r'x.*?x')
m = greedy.search(string)
if m:
    print('mohó:', m.group())
m = lazy.search(string)
if m:
    print('lusta:', m.group())

### 3.3. lookahead és lookbehind

Nulla szélességű illesztés tetszőleges mintára. Akkor hasznos, ha csak egy meghatározott környezetben érdekel az illesztés, illetve, ha bizonyos dolgokra nem szeretnénk illeszteni.

* `(?=...)`: *positive lookahead assertion*
* `(?!...)`: *negative lookahead assertion*
* `(?<=...)`: *positive lookbehind assertion*
* `(?<!...)`: *negative lookbehind assertion*

In [ ]:
pat = re.compile(r'[A-Z]\w+(?<!Immanuel) Kant')
kants = set(pat.findall(open('train/kant.txt').read()))
print(kants)

### 3.4. Gyakorlatok

* (*szorgalmi*) Írjunk egy függvényt, ami egy stringről eldönti, hogy a római szám-e! A neve legyen `is_roman_num(string)`, visszatérési értéke pedig boolean ([wiki](https://hu.wikipedia.org/wiki/R%C3%B3mai_sz%C3%A1m%C3%ADr%C3%A1s)).


## 4. Irodalom

* Jeffrey Friedl: *Mastering Regular Expressions.* O'Reilly, 2009. ([link](http://shop.oreilly.com/product/9780596528126.do))
* [Python: Regular Expression HOWTO](https://docs.python.org/3.6/howto/regex.html)
* [Python: a re modul dokumentációja](https://docs.python.org/3.6/library/re.html)

***

In [ ]:
%%bash

# takaritas
rm -f *.txt